In [ ]:
# 9.9.21
# First let's plot the ecoblock graph

In [20]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [21]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
#modelpath = filepath + "004_GB_IEEE123_OPAL.xls"
modelpath = filepath + "ecoblock_impedexcel.xls"

# load data is not actually used in assessment, but the feeder object builds variables to hold load info
loadfolder = filepath
loadpath = filepath + 'ecoblock_sigbuilder.csv'
headerpath = filepath + 'ecoblock_time_header.csv'
load_data = loadpath

#file_name = string specifying name of dot file created when make_graph() is called
file_name = 'ecoblock'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_3'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2021-09-10 16:38:13.713615


In [22]:
# [ESSENTIAL] create feeder object and plot feeder with make_graph func

fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

#print('depths=',depths) # check this is populated, lists how far each node is from substation
#print('depths length=',len(depths))

# print list of first 10 buses in network
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
import csv  # jaimie do you need this?
graph = fin_feeder.network

  File "C:\Users\jaimi\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jaimi\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\jaimi\Anaconda3\lib\asyncio\base_events.py", line 538, in run_forever
    self._run_once()
  File "C:\Users\jaimi\Anaconda3\lib\asyncio\base_events.py", line 1782, in _run_once
    handle._run()
  File "C:\Users\jaimi\Anaconda3\lib\asyncio\events.py", line 88, in _r

Finished initializing feeder
bus_1
bus_2
bus_3
bus_4
bus_5
bus_6
bus_7
bus_8
bus_9
bus_3001


In [28]:
# mark actuators on graph
lst_act_locs=['bus_4','bus_3079','bus_3089','bus_3011','bus_3015','bus_3025','bus_3029','bus_3033','bus_3037','bus_3038','bus_3044','bus_3049','bus_3056','bus_3058','bus_3057','bus_3059','bus_3063','bus_3062','bus_3001','bus_3008']
#[4 3079 3089 3011 3015 3025 3029 3033 3037 3038 3044 3049 3056 3058 3057 3059 3063 3062 3001 3008];
vis.markActuatorConfig(lst_act_locs, fin_feeder, file_name)

In [ ]:
# # gather impedance info Xii for each actuator
# lst_act_locs=['bus_4','bus_3079','bus_3089','bus_3011','bus_3015','bus_3025','bus_3029','bus_3033','bus_3037','bus_3038','bus_3044','bus_3049','bus_3056','bus_3058','bus_3057','bus_3059','bus_3063','bus_3062','bus_3001','bus_3008']

# Xii_vec=np.array([])
# for cby_load in lst_act_locs:
#     Zii=imp.get_total_impedance_from_substation(fin_feeder, cby_load, depths) # returns dict of 3x3 numpy array for Z_ii
#     Xii=np.real(Zii)
#     print('Xii of ',cby_load,' = ',Xii)
#     #Xii_vec=np.append(Xii_vec,Xii.mean()) # take mean of X_{3x3}

In [19]:
# set config using lst_act_locs, and call Gdisc kgain design func. The func does:
# for 3ph system there are r phase-actuators, for example node6_phA; thus there are r nonzero F elements
# for each of nonzero element of F, det Gdisc conditions; conditions give an upper bound on that F ele search space 
# compute vector of F_ub, dimension rx1
# search space of [0 F_ub] in r-space and det evals of closed-loop system
# Select combo with smallest spectral radius, and return kgains of dim rx1


Xii of  bus_4  =  [[0.2592     0.08830699 0.0894965 ]
 [0.08830699 0.2592     0.08527047]
 [0.0894965  0.08527047 0.2592    ]]
Xii of  bus_3079  =  [[0.2592     0.08830699 0.0894965 ]
 [0.08830699 0.2592     0.08527047]
 [0.0894965  0.08527047 0.2592    ]]
Xii of  bus_3089  =  [[0.2592     0.08830699 0.0894965 ]
 [0.08830699 0.2592     0.08527047]
 [0.0894965  0.08527047 0.2592    ]]
Xii of  bus_3011  =  [[0.3456     0.11774266 0.11932867]
 [0.11774266 0.3456     0.11369396]
 [0.11932867 0.11369396 0.3456    ]]
Xii of  bus_3015  =  [[0.3456     0.11774266 0.11932867]
 [0.11774266 0.3456     0.11369396]
 [0.11932867 0.11369396 0.3456    ]]
Xii of  bus_3025  =  [[0.3456     0.11774266 0.11932867]
 [0.11774266 0.3456     0.11369396]
 [0.11932867 0.11369396 0.3456    ]]
Xii of  bus_3029  =  [[0.3456     0.11774266 0.11932867]
 [0.11774266 0.3456     0.11369396]
 [0.11932867 0.11369396 0.3456    ]]
Xii of  bus_3033  =  [[0.3456     0.11774266 0.11932867]
 [0.11774266 0.3456     0.11369396]


In [17]:
# print kgains vector to csv, and in ephasorsim we read in csv to assign to Kp_vmag, Ki_vmag, etc.
# kgains has dimension rx1, where is number of phase-actuators)

foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
[0.14486088 0.14486088 0.14486088 0.19314784 0.19314784 0.19314784
 0.19314784 0.19314784 0.19314784 0.19314784 0.2414348  0.2414348
 0.2414348  0.2414348  0.2414348  0.2414348  0.2414348  0.2414348
 0.2414348  0.2414348 ]
